In [4]:
import numpy as np
import pandas as pd
from pprint import pprint
import yfinance as yf
from scipy.optimize import minimize

In [5]:
tickers=["AAPL","MSFT","AMZN","GOOGL"]
num_stocks=len(tickers)

cl_price=pd.DataFrame()

In [6]:
for ticker in tickers:
    cl_price[ticker]=yf.download(ticker,period="1y",interval="1d")["Adj Close"]

cl_price.to_csv("Closing_price.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
returns = cl_price.pct_change().dropna()
expected_returns = returns.mean()*252
covariance_matrix = returns.cov()*252
initial_weights = np.array([1.0 / num_stocks] * num_stocks)

def portfolio_risk(weights):
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return portfolio_variance

def sharpe_ratio(weights):
    p_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    p_ret = np.sum((expected_returns * weights) * 252)
    return -p_ret/p_risk

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for asset in range(num_stocks))
result = minimize(sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = result.x
portfolio_expected_return = np.dot(expected_returns, optimal_weights)
portfolio_risk = np.sqrt(np.dot(optimal_weights.T, np.dot(covariance_matrix, optimal_weights)))

print("Optimal Asset Allocations:")
for i, symbol in enumerate(tickers):
    print(f"{symbol}: {optimal_weights[i]:.4f}")

print(f"Expected Portfolio Return: {portfolio_expected_return:.4f}")
print(f"Portfolio Risk (Standard Deviation): {portfolio_risk:.4f}")







Optimal Asset Allocations:
AAPL: 0.1916
MSFT: 0.6111
AMZN: 0.1973
GOOGL: 0.0000
Expected Portfolio Return: 0.4122
Portfolio Risk (Standard Deviation): 0.2329
